Реализуйте mapper первой mapreduce задачи для расчета TF-IDF с помощью Hadoop Streaming.

Формат входных данных следующий: каждая строка содержит номер документа и строку из него, разделенные ":". Ключ выходных данных является составным: он содержит слово документа и его номер, разделенные "#". 

Слово в документе - последовательность символов (букв или цифр), не содержащая пробельных символов и знаков пунктуации.

Sample Input:

    1:1abc:2abc:3abc
    2:1bcd::2bcd::3bcd::
    3:1cde:::2cde:::3cde:::
    34:t0t1:g5_6:::::  :4=1

Sample Output:

    1abc#1	1
    2abc#1	1
    3abc#1	1
    1bcd#2	1
    2bcd#2	1
    3bcd#2	1
    1cde#3	1
    2cde#3	1
    3cde#3	1
    t0t1#34	1
    g5#34	1
    6#34	1
    4#34	1

In [38]:
with open('data/data13.txt', 'r') as f:
    
    import re


    word_pattern = re.compile(r'\w+')

    for line in f:

        key, value = line.strip().split(':', maxsplit=1)
        for match in word_pattern.finditer(value):
            print(f'{value[match.start():match.end()]}#{key}\t1')

1abc#1	1
2abc#1	1
3abc#1	1
1bcd#2	1
2bcd#2	1
3bcd#2	1
1cde#3	1
2cde#3	1
3cde#3	1
t0t1#34	1
g5_6#34	1
4#34	1
1#34	1


---

Реализуйте reducer первой mapreduce задачи для расчета TF-IDF с помощью Hadoop Streaming.

Ключ входных данных составной: он содержит слово и номер документа через "#".

Ключом в выходных данных является слово, а значение состоит из двух элементов, разделенных табуляцией: номер документа и tf (сколько раз данное слово встретилось в данном документе).

Sample Input:

    aut#1	1
    aut#1	1
    aut#1	1
    aut#1	1
    aut#2	1
    aut#2	1
    bene#2	1
    de#2	1
    mortuis#2	1
    nihil#1	1
    nihil#2	1
    Caesar#1	1
    
Sample Output:

    aut	1	4
    aut	2	2
    bene	2	1
    de	2	1
    mortuis	2	1
    nihil	1	1
    nihil	2	1
    Caesar	1	1

In [47]:
with open('data/data14.txt', 'r') as f:
    
    last_key, _sum = None, 0
    
    for line in f:
        
        key, value = line.strip().split()
        
        if last_key and last_key != key:
            term, id_doc = last_key.split('#')
            print(f'{term}\t{id_doc}\t{_sum}')
            last_key, _sum = key, 1
        else:
            last_key, _sum = key, _sum+1
            
    if last_key:
        term, id_doc = last_key.split('#')
        print(f'{term}\t{id_doc}\t{_sum}')

aut	1	4
aut	2	2
bene	2	1
de	2	1
mortuis	2	1
nihil	1	1
nihil	2	1
Caesar	1	1


---

Реализуйте mapper ﻿второй mapreduce задачи для расчета TF-IDF с помощью Hadoop Streaming.

Во входных данных ключом является слово, а значение состоит из номера документа и tf, разделенных табуляцией.

Значение в выходных данных - это тройка: номер документа, tf и единица, разделенные ";".

Sample Input:

    aut	1	4
    aut	2	2
    bene	2	1
    de	2	1
    mortuis	2	1
    nihil	1	1
    nihil	2	1
    Caesar	1	1
    
Sample Output:

    aut	1;4;1
    aut	2;2;1
    bene	2;1;1
    de	2;1;1
    mortuis	2;1;1
    nihil	1;1;1
    nihil	2;1;1
    Caesar	1;1;1

In [48]:
with open('data/data15.txt', 'r') as f:
    
    for line in f:
        
        term, id_doc, tf = line.strip().split()
        print(f'{term}\t{id_doc};{tf};1')

aut	1;4;1
aut	2;2;1
bene	2;1;1
de	2;1;1
mortuis	2;1;1
nihil	1;1;1
nihil	2;1;1
Caesar	1;1;1


---

Реализуйте reducer второй mapreduce задачи для расчета TF-IDF с помощью Hadoop Streaming.

Входные данные: ключ - слово, значение - тройка: номер документа, tf слова в документе и 1 (разделены ';').

Выходные данные: ключ - пара: слово, номер документа (разделены '#'), значение - пара: tf слова в документе, n - количество документов с данным словом (разделены табуляцией).

Sample Input:

    aut	1;4;1
    aut	2;2;1
    bene	2;1;1
    de	2;1;1
    mortuis	2;1;1
    nihil	1;1;1
    nihil	2;1;1
    Caesar	1;1;1
    
Sample Output:

    aut#1	4	2
    aut#2	2	2
    bene#2	1	1
    de#2	1	1
    mortuis#2	1	1
    nihil#1	1	2
    nihil#2	1	2
    Caesar#1	1	1

In [55]:
with open('data/data16.txt', 'r') as f:
    
    lst = []
    last_key = None

    for line in f:
        key, value = line.strip().split()
        id_doc, tf, _ = value.split(';')

        if last_key and last_key != key:
            for i_key, i_id_doc, i_tf in lst:
                print(f'{i_key}#{i_id_doc}\t{i_tf}\t{len(lst)}')
            del lst[:]
            lst.append((key, id_doc, tf))
        else:
            lst.append((key, id_doc, tf))

        last_key = key

    if last_key:
        for i_key, i_id_doc, i_tf in lst:
            print(f'{i_key}#{i_id_doc}\t{i_tf}\t{len(lst)}')
        

aut#1	4	2
aut#2	2	2
bene#2	1	1
de#2	1	1
mortuis#2	1	1
nihil#1	1	2
nihil#2	1	2
Caesar#1	1	1
